In [ ]:
from squlearn.encoding_circuit import QiskitEncodingCircuit
from squlearn.observables import SummedPaulis
from squlearn.optimizers import Adam, SGLBO
from squlearn.qnn import QNNRegressor
from squlearn import Executor
from squlearn.qnn import SquaredLoss

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

import matplotlib.pyplot as plt

## PQC

In [ ]:
qc = QuantumCircuit(8)
p = ParameterVector('p', 24)
x = ParameterVector('x', 1)
num_parameters = 24

p2 = np.array([-0.13005136, -1.34331017, 2.44392299, 1.64405423, -0.36315523, 0.45344925,
                   0.18577077, -0.13904446])

qc.rx(1 * np.arccos(x[0]), 0)
qc.rx(2 * np.arccos(x[0]), 1)
qc.rx(3 * np.arccos(x[0]), 2)
qc.rx(4 * np.arccos(x[0]), 3)
qc.rx(1 * np.arccos(x[0]), 4)
qc.rx(2 * np.arccos(x[0]), 5)
qc.rx(3 * np.arccos(x[0]), 6)
qc.rx(4 * np.arccos(x[0]), 7)

qc.ry(p[0], 0)
qc.ry(p[1], 1)
qc.ry(p[2], 2)
qc.ry(p[3], 3)
qc.ry(p[4], 4)
qc.ry(p[5], 5)
qc.ry(p[6], 6)
qc.ry(p[7], 7)

qc.cx(0, 1)
qc.cx(2, 3)
qc.cx(4, 5)
qc.cx(6, 7)
qc.cx(1, 2)
qc.cx(3, 4)
qc.cx(5, 6)
qc.cx(7, 0)

qc.rx(1 * np.arccos(x[0]), 0)
qc.rx(2 * np.arccos(x[0]), 1)
qc.rx(3 * np.arccos(x[0]), 2)
qc.rx(4 * np.arccos(x[0]), 3)
qc.rx(1 * np.arccos(x[0]), 4)
qc.rx(2 * np.arccos(x[0]), 5)
qc.rx(3 * np.arccos(x[0]), 6)
qc.rx(4 * np.arccos(x[0]), 7)

qc.ry(p[8], 0)
qc.ry(p[9], 1)
qc.ry(p[10], 2)
qc.ry(p[11], 3)
qc.ry(p[12], 4)
qc.ry(p[13], 5)
qc.ry(p[14], 6)
qc.ry(p[15], 7)


qc.cx(0, 1)
qc.cx(2, 3)
qc.cx(4, 5)
qc.cx(6, 7)
qc.cx(1, 2)
qc.cx(3, 4)
qc.cx(5, 6)
qc.cx(7, 0)

qc.rx(1 * np.arccos(x[0]), 0)
qc.rx(2 * np.arccos(x[0]), 1)
qc.rx(3 * np.arccos(x[0]), 2)
qc.rx(4 * np.arccos(x[0]), 3)
qc.rx(1 * np.arccos(x[0]), 4)
qc.rx(2 * np.arccos(x[0]), 5)
qc.rx(3 * np.arccos(x[0]), 6)
qc.rx(4 * np.arccos(x[0]), 7)

qc.ry(p[16], 0)
qc.ry(p[17], 1)
qc.ry(p[18], 2)
qc.ry(p[19], 3)
qc.ry(p[20], 4)
qc.ry(p[21], 5)
qc.ry(p[22], 6)
qc.ry(p[23], 7)


qc.cx(0, 1)
qc.cx(2, 3)
qc.cx(4, 5)
qc.cx(6, 7)
qc.cx(1, 2)
qc.cx(3, 4)
qc.cx(5, 6)
qc.cx(7, 0)

pqc = QiskitEncodingCircuit(qc)
pqc.draw("mpl")

In [ ]:
## Observable

In [ ]:
nqubits = 8
op = SummedPaulis(nqubits)

In [ ]:
# Randomly initialize parameters of the encoding circuit
np.random.seed(13)
param_ini = np.random.rand(pqc.num_parameters)
# Initialize parameters of the observable as ones
param_op_ini = np.ones(op.num_parameters)

## SGLBO

In [ ]:

#x0 = [[i * 0.02] for i in range(15)]
x0 = [[1e-6], [2e-6], [3e-6], [4e-6], [5e-6], [6e-6], [7e-6], [8e-6], [9e-6], [1e-5], [2e-5], [3e-5], [4e-5], [5e-5], [6e-5], [7e-5], [8e-5], [9e-5], [1e-4], [2e-4], [3e-4], [4e-4], [5e-4], [6e-4], [7e-4], [8e-4], [9e-4], [1e-3], [2e-3], [3e-3], [4e-3], [5e-3], [6e-3], [7e-3], [8e-3], [9e-3], [1e-2], [2e-2], [3e-2], [4e-2], [5e-2], [6e-2], [7e-2], [8e-2], [9e-2], [1e-1], [2e-1]]
optimizer_options = {"bo_aqc_func": "EI", "bo_aqc_optimizer": "lbfgs", "bo_bounds": [(0.0, 0.2)], "log_file": f"sglbo_{num_parameters}_params_new.log", "bo_n_calls": 60, "bo_x0_points": x0, "maxiter": 500}

qnn_simulator_sglbo = QNNRegressor(
        pqc,
        op,
        Executor("statevector_simulator"),
        SquaredLoss(),
        SGLBO(optimizer_options),
        param_ini,
        param_op_ini=param_op_ini,
        opt_param_op=True,  # Keine Observablen optimierung
        parameter_seed=124
    )

## Fit

In [ ]:
# Data that is inputted to the QNN
x_train = np.arange(-0.5, 0.6, 0.1)
# Data that is fitted by the QNN
y_train = np.sin(6.0 * x_train)

qnn_simulator_sglbo.fit(x_train, y_train)